In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import torchvision.transforms as transforms
from PIL import Image
import cv2

class PedestrianDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.dataframe = dataframe
        self.transform = transform
        self.label_mapping = {"Low": 0, "Medium": 1, "High": 2}

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_path = self.dataframe.iloc[idx]["unlabeled_image_path"]
        label = self.label_mapping[self.dataframe.iloc[idx]["risk_level"]]

        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = Image.fromarray(image)

        if self.transform:
            image = self.transform(image)

        return image, torch.tensor(label)

def load_datasets(csv_path, batch_size=32):
    df = pd.read_csv(csv_path).sample(frac=1).reset_index(drop=True)
    df = df.drop(["labeled_image_path", "pedestrian_pixels"], axis=1)
    
    train_df = df[:int(0.8 * len(df))]
    val_df = df[int(0.8 * len(df)):int(0.9 * len(df))]
    test_df = df[int(0.9 * len(df)):]

    transform = transforms.Compose([
        transforms.Resize((128, 128)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
    ])

    train_dataset = PedestrianDataset(train_df, transform=transform)
    val_dataset = PedestrianDataset(val_df, transform=transform)
    test_dataset = PedestrianDataset(test_df, transform=transform)

    return (
        DataLoader(train_dataset, batch_size=batch_size, shuffle=True),
        DataLoader(val_dataset, batch_size=batch_size, shuffle=False),
        DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    )

In [2]:
train_loader, val_loader, test_loader = load_datasets("pedestrian_risk_analysis.csv")

In [3]:
import torch
import torch.nn as nn

In [4]:
class PedestrianCNN(nn.Module):
    def __init__(self):
        super(PedestrianCNN, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.fc_layers = nn.Sequential(
            nn.Linear(128 * 16 * 16, 128),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(128, 3)  
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = x.view(x.size(0), -1)
        x = self.fc_layers(x)
        return x

def get_model(device):
    model = PedestrianCNN().to(device)
    return model

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [7]:
import torch
import torch.optim as optim

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = PedestrianCNN().to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

_loss_plot_test = []
num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    total_loss, correct, total_samples = 0, 0, 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = loss_fn(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        correct += (outputs.argmax(dim=1) == labels).sum().item()
        total_samples += labels.size(0)

    avg_train_loss = total_loss / len(train_loader)
    train_accuracy = correct / total_samples

    model.eval()  
    val_loss, val_correct, val_samples = 0, 0, 0

    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = loss_fn(outputs, labels)
            val_loss += loss.item()
            val_correct += (outputs.argmax(dim=1) == labels).sum().item()
            val_samples += labels.size(0)

    avg_val_loss = val_loss / len(val_loader)
    val_accuracy = val_correct / val_samples
    _loss_plot_test.append(avg_val_loss)

    print(f"Epoch [{epoch+1}/{num_epochs}] | "
          f"Train Loss: {avg_train_loss:.4f}, Train Acc: {train_accuracy:.4f} | "
          f"Val Loss: {avg_val_loss:.4f}, Val Acc: {val_accuracy:.4f}")

Epoch [1/10] | Train Loss: 0.7205, Train Acc: 0.6657 | Val Loss: 0.5166, Val Acc: 0.7660
Epoch [2/10] | Train Loss: 0.4634, Train Acc: 0.8047 | Val Loss: 0.3040, Val Acc: 0.8914
Epoch [3/10] | Train Loss: 0.3581, Train Acc: 0.8535 | Val Loss: 0.2645, Val Acc: 0.8979
Epoch [4/10] | Train Loss: 0.3010, Train Acc: 0.8754 | Val Loss: 0.2307, Val Acc: 0.9136
Epoch [5/10] | Train Loss: 0.2640, Train Acc: 0.8927 | Val Loss: 0.2413, Val Acc: 0.9099
Epoch [6/10] | Train Loss: 0.2328, Train Acc: 0.8986 | Val Loss: 0.1991, Val Acc: 0.9294
Epoch [7/10] | Train Loss: 0.2152, Train Acc: 0.9091 | Val Loss: 0.1871, Val Acc: 0.9257
Epoch [8/10] | Train Loss: 0.2004, Train Acc: 0.9168 | Val Loss: 0.1856, Val Acc: 0.9090
Epoch [9/10] | Train Loss: 0.1853, Train Acc: 0.9204 | Val Loss: 0.1944, Val Acc: 0.9183
Epoch [10/10] | Train Loss: 0.1758, Train Acc: 0.9274 | Val Loss: 0.1827, Val Acc: 0.9285


In [9]:
#save 
torch.save({
    'epoch': num_epochs,
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'loss': loss,
}, "checkpoint.pth")

In [ ]:
# resume training ? (double check this)
checkpoint = torch.load("checkpoint.pth")
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']